# Lab 6 - Face Recognition
In this lab we perform face recognition using PCA and KNN. We will also reuse some of the face detection work we completed in the previous face detection notebook.

By the end of this lab, you should be able to:
* Use PCA and KNN for facial recognition
* Use both Face Detection and Face Recognition in a single pipeline

## Packages
In this lab we will be using the following packages:
* *OpenCV* for face detection
* *numpy* for interacting with image data
* *sklearn* for PCA, KNN and metric computation
* *matplotlib* for visualization

In [ ]:
import math

import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from urllib.request import urlopen

Refer to the `Packages` notebook for more information on packages we have used before.

In this lab, we will reuse some of the functions we created previously. They are provided below for your convenience.

In [ ]:
# General Functions

def display_image(image, title=None):
    fig, axes = plt.subplots(figsize=(12, 8))

    if image.ndim == 2:
        axes.imshow(image, cmap='gray', vmin=0, vmax=255)
    else:
        axes.imshow(image)

    if title is not None:
        plt.title(title)

    plt.show()


def load_image_from_url(url):
    """Given a URL, loads the image into a numpy

    Image loaded in RGB, with HWC channel ordering

    Args:
        url (str): The URL of the image to load

    Returns:
        (np.ndarray): The RGB, HWC ordered image
    """
    with urlopen(url) as ur:
        image = np.asarray(bytearray(ur.read()), dtype='uint8')
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image


def draw_rectangles_tlbr(image, rectangles, colour=(255, 0, 0), thickness=1):
    """Draws a list of rectangles in [tlx, tly, brx, bry] form onto an image

    Args:
        image (np.ndarray): The image to overlay rectangles on
        rectangles (list of list/np.ndarray): A list of rectangles to overlay on the image.
            rectangles should be in the form: [tlx, tly, brx, bry].
        colour (3-tuple): The RGB colour to draw boxes in
        thickness (int): The thickness of the rectangles

    Returns:
        (np.ndarray): A copy of the image with all rectangles overlaid
    """
    # Copy the image to not mutate the original image
    image = image.copy()

    # Draw rectangles
    for rectangle in rectangles:
        tlx, tly, brx, bry = rectangle.astype(np.int32)
        cv2.rectangle(
            image, (tlx, tly), (brx, bry),
            color=colour, thickness=thickness)

    return image


# Haar Cascades

def preprocess_image_haar(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)


def detect_faces_haar(image, haar_cascade_classifier, scale_factor, min_neighbours):
    gray_image = preprocess_image_haar(image)
    detections = haar_cascade_classifier.detectMultiScale(gray_image, scaleFactor=scale_factor, minNeighbors=min_neighbours)
    if isinstance(detections, np.ndarray):
        detections[:, 2] = detections[:, 0] + detections[:, 2]
        detections[:, 3] = detections[:, 1] + detections[:, 3]
    return detections

We will make use of a Haar cascade classifier in section 2.4 of this lab. For your convenience we provide the necessary code to create a Haar cascade classifier instance.

In [ ]:
xml_filepath = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
haar_cascade_classifier = cv2.CascadeClassifier(xml_filepath)

# 1. Face Recognition
Face recognition is the process of assigning an identity to a given image of a face.

In this section we will look at using Principal Component Analysis (PCA) and applying the K-Nearest Neighbours (KNN) algorithm to perform face recognition in images.  
Please refer to your lecture material for an in depth discussion on PCA.

## 1.1 Dataset

To be able to perform face recognition, we will first need a labelled dataset of faces which includes their identities. This will enable us to fit our PCA and KNN algorithm to recognize images of faces on new data.

Like earlier in this lab, we will make use of a subset of data from the [Labelled Faces in the Wild dataset](http://vis-www.cs.umass.edu/lfw/). This subset differs from the previous one we used in that we have many more images, the resolution of these images is larger, and that these images also contain a label representing their identity.

In the code cell below we download the dataset and look at some properties of the data. Note that when downloading the dataset we specify that we want grayscale images.

It may take a few moments for this code cell to run the first time you execute it, as the dataset takes some time to download.

In [ ]:
lfw_people_dataset = fetch_lfw_people(min_faces_per_person=70, resize=0.4, color=False)
print(f'The type of the dataset is: {type(lfw_people_dataset)}')
print(f'The dataset has properties: {list(lfw_people_dataset.keys())}')

We see above that the dataset is of type `sklearn.utils.Bunch`. This is a specific type of object that has various properties attached to it that we can access.

From the print out above, we can see there are 5 different properties that this dataset has. A description of each of these is as follows:
* `data`: A *(N, D)* *numpy* array with image data collapsed into a single dimension.
* `images`: A *(N, H, W)* *numpy* array representing the image data
* `target`: A *(N, )* *numpy* array representing the labels belonging to each image (We will look at this further soon)
* `target_names`: A list of length *N* representing the name belonging to each image
* `DESCR`: A text description of the dataset

Where:
* `N`: Represents the number of images in the dataset
* `H`: Represents the height of images in the dataset
* `W`: Represents the width of images in the dataset
* `D`: Represents the number of dimensions per-image (i.e. `D` = `H` * `W`)

The `data` and `images` properties currently represent pixels using values in the 0--1 range. We will now modify these values to range from 0--255 instead.

In [ ]:
if lfw_people_dataset.data.max() <= 1:
    lfw_people_dataset.data *= 255

if lfw_people_dataset.images.max() <= 1:
    lfw_people_dataset.images *= 255

Let's first start by printing out the `target_names` to see who is in our dataset.

In [ ]:
print(f'The people in our dataset are: {lfw_people_dataset.target_names}')

It looks like there are 7 people in this dataset.  

Next, let's inspect the images and target data in the dataset

In [ ]:
print(f'The shape of all images in our dataset is: {lfw_people_dataset.images.shape}')
print(f'The shape of the targets in our dataset is: {lfw_people_dataset.target.shape}')
print('-' * 50)
print(f'The shape of the first image in our dataset is: {lfw_people_dataset.images[0].shape}')
print(f'The target for the first image in our dataset is: {lfw_people_dataset.target[0]}')

After running the above code cell, it should be quite clear that we have 1288 images in our dataset, with each image having a height and width of 50px and 37px respectively. That means that each image in our dataset can be flattened into a 1850 (50 * 37) dimensional vector.

We can also see that the target of our first image is the integer value 5. To find the name of who this person is, we use this value as an index into the list of `target_names`.

**Task**: In the code cell below, extract the first image and target value from the dataset. Convert the image to a `np.uint8` datatype, find the name associated to the target value, then display the image with the title set to the name of who is in the image.

In [ ]:
# TODO: Extract the first image from the dataset



# TODO: Extract the target value from the dataset



# TODO: Find the name associated to the target value



# TODO: Display the image with the appropriate title




In [ ]:
#@title Task solution

# TODO: Extract the first image from the dataset
image = lfw_people_dataset.images[0]


# TODO: Extract the target value from the dataset
target_value = lfw_people_dataset.target[0]


# TODO: Find the name associated to the target value
target_name = lfw_people_dataset.target_names[target_value]


# TODO: Display the image with the appropriate title
display_image(image, target_name)

## 1.2 Dimensionality Reduction with Principal Component Analysis (PCA)

We have our dataset, but how can we recognize who the faces belong to?

As it stands, each image in our dataset is 50x37px, which when flattened, results in a 1850 dimensional vector. We could immediately try to use this vector for face recognition, however there will be a lot of redundant information in that vector, and it would be quite computationally expensive to use. A better approach would be to try reduce the dimensionality of the data, whilst preserving important information, then using the smaller data dimensionality representation for face recognition.

Principal Component Analysis is well suited for performing dimensionality reduction on our data, and can do so in a fully unsupervised manner. This means it can work out how to reduce the dimensionality of our data whilst still preserving important information in a data-driven approach.

This section focusses on fitting PCA to our dataset and does not discuss in detail how PCA works. Refer to your lecture notes for a detailed discussion of PCA.

To apply PCA on our data, we first need to fit PCA to our dataset so that it can determine how to reduce the dimensionality of our data whilst preserving important information.

To do this, let's first extract the set of input data and corresponding targets from our dataset. The set of input data will be an (N, D) dimensional array, and the targets will be an (N, ) dimensional vector.

**Task**: Extract the inputs and targets from the dataset.

In [ ]:
# TODO: Extract the inputs from the dataset
# inputs = ...


# TODO: Extract the targets from the dataset
# targets = ...


# Check the shape of the inputs and targets
print(inputs.shape)        # Should be: (1288, 1850)
print(targets.shape)       # Should be: (1288, )

In [ ]:
#@title Task solution

# TODO: Extract the inputs from the dataset
inputs = lfw_people_dataset.data


# TODO: Extract the targets from the dataset
targets = lfw_people_dataset.target


# Check the shape of the inputs and targets
print(inputs.shape)
print(targets.shape)

To ensure we can properly evaluate our face recognition system, we should split our data up into a training and testing set. This is extremely common to do when training any machine learning algorithm as it ensures we can do fair evaluation on data that our algorithm has not used during training.

**Task**: In the code cell below we split the inputs and targets into a training and testing set. Print the shape of each train/test input/target.

In [ ]:
# Split dataset into train/test (We will use an existing function)
train_input, test_input, train_target, test_target = train_test_split(inputs, targets, test_size=0.25, random_state=42)

# TODO: Print the shape of train/test input/target



In [ ]:
#@title Task solution

# Split dataset into train/test (We will use an existing function)
train_input, test_input, train_target, test_target = train_test_split(inputs, targets, test_size=0.25, random_state=42)

# TODO: Print the shape of train/test input/target
print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

You should see that we now have 966 images in our training set and 322 images in our testing set. We will fit our PCA using the 966 images in the training set.

The next thing we need to do is decide how many principal components we want to reduce our data to. That is, our data currently has 1850 dimensions, so how many dimensions do we want to reduce that to.

Let's choose 100. That is, we want our image data to be described by a 100 dimensional vector.

In [ ]:
NUM_COMPONENTS = 100

Now we know how many components we want to reduce our data to, the next step is to setup PCA and fit our training set of input data!

**Task**: With reference to the [PCA documentation](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html), create an instance of PCA, then fit it to the `train_input` data.

In [ ]:
# TODO: Create an instance of PCA
# pca = ...


# TODO: Fit PCA to the training data




In [ ]:
#@title Task solution

# TODO: Create an instance of PCA
pca = PCA(n_components=NUM_COMPONENTS)


# TODO: Fit PCA to the training data
pca.fit(train_input)

What did that just do?

By fitting PCA to our training dataset, our `pca` object can now reduce high dimensional image data to a relatively lower 100 dimensional vector. The value of each dimension corresponds to the contribution of each eigenface that can sum up to reproduce the original image.

Let's explore this to develop a better understanding of what has occurred.

In the code cell below, we display the first 12 eigenfaces from our PCA (That is, the first 12 components of our fitted PCA).

In [ ]:
# Get the eigenfaces from components of pca (We reshape these to be (100x50x37 dimensional))
eigenfaces = pca.components_.reshape((NUM_COMPONENTS, 50, 37))

# Extract the first 12 eigenfaces
eigenfaces = eigenfaces[:12]

# Display the eigenfaces
fig, axs = plt.subplots(3, 4, figsize=(19.2, 10.8))
axs = np.ravel(axs)
for idx, eigenface in enumerate(eigenfaces):
    axs[idx].imshow(eigenface, cmap=plt.cm.gray)
    axs[idx].set_title(f'Eigenface {idx}')

plt.show()

We can also use these eigenfaces to approximately reconstruct an original image.

In the code cell below we extract the first image from the original dataset and transform it to the lower dimensionality space. We then attempt to reconstruct it using a number of eigenfaces. At the end of this code cell we display both the original and reconstructed images for visual comparison. Experiment with changing the number of eigenfaces used to reconstruct the image to see how many are required to produce a reasonable reconstruction.

In [ ]:
# Set how many eigenfaces we want to use to reconstruct the original image
USE_N_EIGENFACES = 10

# Extract the first image from the dataset
first_image = lfw_people_dataset.images[0]
print(f'The shape of this image is: {first_image.shape}')

# Flatten the image to produce a vector, then transform it to the lower dimensionality space
#    We reshape to (1, -1) which gives us data with dimensionality: (1, 1850)
reduced_image = pca.transform(first_image.reshape(1, -1))
print(f'The shape of this image with reduced dimensionality is: {reduced_image.shape}')

# Extract the first N eigenvalues
eigenvalues = reduced_image[0, :USE_N_EIGENFACES]

# Print out the eigenvalues
print(f'The eigenvalues we will use to reconstruct the image are: {eigenvalues}')

# Extract the first N eigenfaces (Don't reshape as we need to combine with eigenvalues)
eigenfaces = pca.components_[:USE_N_EIGENFACES]
print(f'Using the first {USE_N_EIGENFACES} for image reconstruction')

# Create the reconstructed image by combining eigenvalues and eigenfaces
#    We also need to add the mean image to do this reconstruction
reconstructed_image = np.dot(eigenvalues, eigenfaces) + pca.mean_
reconstructed_image = reconstructed_image.reshape((50, 37))

# Display the original image and reconstructed image
display_image(first_image, 'Original Image')
display_image(reconstructed_image, f'Reconstructed Image using first {USE_N_EIGENFACES} eigenfaces')

You might see that using only the first 10 eigenvectors/eigenfaces doesn't really reproduce the image too well. As you bump this number up, you should see better resemblance between the original and reconstructed images.

What is really impressive though is that now we can represent our original 50x37px images (with a total of 1850 values) by only 100 numbers!

Now we have a bit of a better understanding on exactly *what* PCA did to our data, the next step is to transform all of our original training and testing data from their original dimensionality to the reduced dimensionality space.

**Task**: In the below cell, transform `train_input` and `test_input` using PCA. Store these in variables `train_input_pca` and `test_input_pca` respectively. You saw an example of how we can do this in the previous code cell, alternatively refer to the PCA documentation.

In [ ]:
# TODO: Transform train_input and test_input into the lower dimensionality space with PCA



# Test your solution
print(train_input_pca.shape)     # Should be: (966, 100)
print(test_input_pca.shape)      # Should be: (322, 100)

In [ ]:
#@title Task solution

# TODO: Transform train_input and test_input into the lower dimensionality space with PCA
train_input_pca = pca.transform(train_input)
test_input_pca = pca.transform(test_input)

# Test your solution
print(train_input_pca.shape)     # Should be: (966, 100)
print(test_input_pca.shape)      # Should be: (322, 100)

## 1.3 Face Recognition with K-Nearest Neighbour (KNN)

We've successfully been able to reduce the dimensionality of our original image data, so the next thing we need to do is somehow use this reduced dimensionality representation to take a new image of a face and determine the identity of the face.

To do this we will be using the K-Nearest Neighbour algorithm.

<details>
<summary style='cursor:pointer;'><u>Expand for KNN background</u></summary>

K-Nearest Neighbour (KNN) is a machine learning classification algorithm that can classify a new piece of data based on an initial training set of data.
    
The process for KNN classification is:
* Take a set of N-dimensional labelled data to initialize the algorithm
* Given a new N-dimensional data to be classified:
    * Compute the distance between the data to be classified and each example used to initialize the algorithm
    * Based on the value of K (The number of neighbours), assign a classification label by voting based on the distance between the new data point and the labels of the K nearest data points
    
Given we used PCA to reduce the dimensionality of our dataset, our data is 100 dimensional.
    
You can visualize an example of KNN by looking at this useful [online KNN Demo](http://vision.stanford.edu/teaching/cs231n-demos/knn/). In this demo, the data is 2-dimensional (which enables us to visualize it on a plane).
</details>

When using KNN, we are able to choose the value of K (the number of neighbours). Let's set this to 6.

In [ ]:
NUM_NEIGHBOURS = 6

Now we've defined the number of neighbours we want, the next step is to setup our KNN algorithm and initialize it with our training set of input data and labels!

**Task**: With reference to the [KNN documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), create an instance of KNN, then fit it to the `train_input_pca` and `train_target` data.

In [ ]:
# TODO: Create an instance of KNN
# knn = ...


# TODO: Fit KNN to the training data (inputs and targets)




In [ ]:
#@title Task solution

# TODO: Create an instance of KNN
knn = KNeighborsClassifier(n_neighbors=NUM_NEIGHBOURS)


# TODO: Fit KNN to the training data (inputs and targets)
knn.fit(train_input_pca, train_target)

Great work! It was as easy as that to setup our KNN classifier! Now it's ready for us to perform classification!

Let's test this out on the first example of our dataset (In the next section we will do a proper performance evaluation)

**Task**: In the code cell below, take the first reduced dimensionality example from `test_input_pca` and classify the identity of the face using your KNN classifier. Display the original image along with the predicted label and actual label. Look at the KNN documentation to see how you can use it to make a prediction.

In [ ]:
# TODO: Extract the first example from test_input_pca
# pca_image = ...


# TODO: Reshape your data to get it in a form usable to perform classification
#       KNN expects a set of data to predict, so create an empty dimension
pca_image = pca_image.reshape(1, -1)



# TODO: Predict the label of this example



# TODO: Extract the original image (from test_input) and reshape it back into (50, 37)



# TODO: Extract the actual label (from test_target)



# TODO: Convert the predicted and actual labels into an text name (using lfw_people_dataset.target_names)



# TODO: Display the original image, with the predicted label and actual label in the title




In [ ]:
#@title Task solution

# TODO: Extract the first example from test_input_pca
pca_image = test_input_pca[0]


# TODO: Reshape your data to get it in a form usable to perform classification
#       KNN expects a set of data to predict, so create an empty dimension
pca_image = pca_image.reshape(1, -1)


# TODO: Predict the label of this example
predicted_label = knn.predict(pca_image)


# TODO: Extract the original image (from test_input) and reshape it back into (50, 37)
original_image = test_input[0].reshape(50, 37)


# TODO: Extract the actual label (from test_target)
actual_label = test_target[0]


# TODO: Convert the predicted and actual labels into an text name (using lfw_people_dataset.target_names)
# NOTE: predicted_label is returned as a list of labels. We extract the first label (There is only 1 given we only predicted 1 image)
predicted_label_name = lfw_people_dataset.target_names[predicted_label[0]]
actual_label_name = lfw_people_dataset.target_names[actual_label]


# TODO: Display the original image, with the predicted label and actual label in the title
title = f'Prediction: {predicted_label_name}. Ground Truth: {actual_label_name}. Correct: {predicted_label_name == actual_label_name}'
display_image(original_image, title)

**Comprehension Question**

You have a headshot photo of your face and want to test this face recognition system on your photo. What do you expect would happen?

<details>
<summary style='cursor:pointer;'><u>Answer</u></summary>

This face recognition system is only able to recognize faces that were used in the dataset to initialize KNN, so it would be impossible for it to recognize your face.  
You would expect the KNN classifier to assign a label based on the faces that look most similar to yours from this dataset (Assuming the PCA dimensionality reduction worked well on your image).  
</details>

## 1.4 Evaluation
If everything went well, you should have found the first face in the test set was correctly recognized.

Before finishing this section on Face Recognition, it's very important that we evaluate exactly how well our PCA and KNN face recognition works on our test set! This will let us know if the system we have developed is reliable enough, or if we need to change some parts in our design.

**Task**: In the code cell below, use your KNN instance to make predictions on all examples in `test_input_pca`. Store these predictions in the variable `test_prediction`.

In [ ]:
# TODO: Make predictions on test_input_pca with your KNN instance




In [ ]:
#@title Task solution

# TODO: Make predictions on test_input_pca with your KNN instance
test_prediction = knn.predict(test_input_pca)

In Lab 3 we looked at different evaluation metrics we could use to evaluate the performance of our image classification model, specifically we looked at per-class `precision`, `recall` and `f1 score`.

Within `sklearn` we can produce a classification report that summarizes the precision, recall and f1 scores per-class and also gives us an averaged score for the whole dataset. Let's produce this report for our dataset!

**Task**: With reference to the [*`classification_report()`* documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html), generate and print out the classification report given the predictions in `test_prediction` and ground truth in `test_target`. Make sure you specify the `target_names` argument.

In [ ]:
# TODO: Generate the classification report
# report = ...


# TODO: Print out the report



In [ ]:
#@title Task solution

# TODO: Generate the classification report
report = classification_report(test_target, test_prediction, target_names=lfw_people_dataset.target_names)


# TODO: Print out the report
print(report)

You should see that the values in the report exactly match what you computed in each of the previous sections.

Overall these results aren't really that great. The unweighted precision and recall of our face recognition was only *0.51* and *0.41* respectively.  

Depending on what this system is being used for, these results may be acceptible, however in general you would ideally want better performance before using this system in a practical application.  
**This is why evaluation is so important when designing any machine learning system.**

We leave it up to you in `Challenge Task 1` to modify the PCA and KNN parameters to see if you can achieve better performance.

# 2. Face Detection and Recognition with Haar Cascades, PCA and KNN

To bring together what we have learned about both face detection and recognition, let's create a system that can take a whole image and detect and recognize faces within that image.

## 2.1 Loading Sample Data

Given our face recognition system was trained on a dataset with only 7 different identities, we need to choose an image that contains someone in that dataset.

In the code cell below we load and display an image of George W. Bush.

In [ ]:
url = 'https://upload.wikimedia.org/wikipedia/commons/c/cf/20081205_George_W_Bush_Economy.jpg'
image = load_image_from_url(url)
display_image(image)

## 2.2 Preprocessing

We fit our face recognition system (PCA and KNN) on data contained in an existing dataset.

Data in that dataset was grayscale with a size of 50x37px. If we want to reuse that face recognition system, it is important that we first preprocess any image data we want to perform image recognition on to the same form.

**Task**: Write a function `preprocess_image_face_recognition` that takes an *image* as an argument (assumed to be in RGB), converts it to grayscale, resizes it to 50x37px, then returns the preprocessed image.

In [ ]:
# TODO: Your function here



# Use this to test your function (You should see a gray image with height 50px and width 37px)
preprocessed_image = preprocess_image_face_recognition(image)
display_image(preprocessed_image)

In [ ]:
#@title Task solution

def preprocess_image_face_recognition(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image = cv2.resize(image, (37, 50), interpolation=cv2.INTER_AREA)
    return image

**Comprehension Question**

With reference to the visualization of the preprocessed image, what issues can you foresee when using it with our face recognition system?

<details>
<summary style='cursor:pointer;'><u>Answer</u></summary>

The dataset we used to fit PCA and KNN contained images of faces that only contained faces. When we preprocessed the whole image, we see that the face is only about 5-10 pixels wide and 10-15 pixels high, with no recognizable face features. This looks entirely different to all examples that were contained in our dataset.  
    
We cannot rely on the identity that our face recognition system assigns this image.
</details>

## 2.3 Recognition
Let's now look at performing face recognition on image data using our PCA/KNN recognition system. To make things easier, we will create a function that can take an image and produce a face identity corresponding to that image.

**Task**: Write a function named `recognize_face` that:
* Takes an *image*, *pca* instance, *knn* instance and a list of *names*
* Preprocesses the image ready for processing through the PCA/KNN recognition system
* Reshapes the image to create an empty dimension (with *.reshape(1, -1)*)
* Reduces the dimensionality of the preprocessed image with the *pca* instance
* Predicts the face identity of the reduced dimensionality image using the *knn* instance
* Converts the predicted label from *knn* into a name, using the list of *names*
* Returns the name associated to the image

In [ ]:
# TODO: Your function here


# Test your solution
name = recognize_face(image, pca, knn, lfw_people_dataset.target_names)
display_image(image, name)

In [ ]:
#@title Task solution

def recognize_face(image, pca, knn, target_names):
    # Preprocess image
    image = preprocess_image_face_recognition(image)

    # Reshape
    image = image.reshape((1, -1))

    # Dimensionality reduction
    pca_image = pca.transform(image)

    # Face recognition
    label = knn.predict(pca_image)

    # Label idx -> name
    name = target_names[label[0]]

    return name

Looks like our system got it wrong!

However, given what the input to our face recognition system looked like, it's not really all that surprising.

How can we fix this so that the input to the recognition system looks better?

## 2.4 Detection and Recognition Pipeline
It turns out that face detection and face recognition can go hand-in-hand to create a single complete pipeline that addresses the issue we ran into above.

What if instead of feeding in the whole image to our face recognition system, we first *detect* faces in the image, extract *crops* of those faces, then use our face recognition system on those face crops?

Let's give this a try!

**Task**: Using your `haar_cascade_classifier`, detect the faces within the `image` of George W. Bush and display the image with face detections overlaid. Use a *1.3* `scaleFactor` and *6* `minNeighbors`.

In [ ]:
# TODO: Your solution here




In [ ]:
#@title Task solution

face_detections = detect_faces_haar(image, haar_cascade_classifier, 1.3, 6)
overlaid_image = draw_rectangles_tlbr(image, face_detections, thickness=3)
display_image(overlaid_image)

So far so good! Our Haar cascade classifier was successfully able to detect the face in the image!

Next, let's crop out the image of the face in preparation for recognition.

**Task**: Using the detected face bounding rectangle from the previous code cell, crop out the image data bounded by that rectangle and display it (Store the cropped image in the variable `face_crop`). Refer to Lab 1 for examples of how to crop an image.

<details>
<summary style='cursor:pointer;'><u>Hint</u></summary>

The bounding rectangle is in the form: top-left, width, height. You will need to transform this into the form: top-left, bottom-right to crop your image. When cropping, these coordinates **must** be integers.
</details>

In [ ]:
# TODO: Your solution here




In [ ]:
#@title Task solution

# Extract the face detection
tlx, tly, brx, bry = face_detections[0]

# Crop the image (Cast variables to integers to ensure integer crop coordinates)
face_crop = image[tly:bry, tlx:brx, :]

# Display the cropped image
display_image(face_crop)

This is looking better! Before performing recognition, let's check what the preprocessed image looks like.

In [ ]:
preprocessed_image = preprocess_image_face_recognition(face_crop)
display_image(preprocessed_image)

This looks infinitely better than the preprocessed whole image! We can actually make out face features in this crop, and now this crop looks much more similar to examples in our dataset.

Let's try recognize the face from this crop!

**Task**: Recognize the face in the `face_crop` image and display the `face_crop` image with the title set to the name of the recognized face.

In [ ]:
# TODO: Your solution here




In [ ]:
#@title Task solution

name = recognize_face(face_crop, pca, knn, lfw_people_dataset.target_names)
display_image(face_crop, name)

If everything worked well, you should see that our face recognition system has now successfully recognized this face as George W. Bush!

This is a great example to show how we can combine multiple computer vision applications together in a single pipeline to produce results that we otherwise would not be able to achieve.

# 3. Additional Tasks

This section contains additional tasks that you should complete within this lab to further your understanding of face detection and recognition.

## 3.1 Multiple Faces Example
As another example for you to pull everything together again, let's try recognize faces in an image with multiple people.

**Task**: In the code cell below:
* Load the image from the given URL (George W. Bush is on the left, Donald Rumsfeld is on the right)
* Display the image
* Detect, overlay and display all faces in the image using your Haar cascade classifier with *1.3* `scaleFactor` and *6* `minNeighbors` *(Refer to Lab 5 - Section 2.4 - Classification)*
* For every detected face:
    * Crop out the face *(Refer to Section 2.4 - Detection and Recognition Pipeline)*
    * Use the face recognition system to determine the name belonging to the face *(Refer to Section 2.3 - Recognition)*
    * Display the face along with the name of who is in the image
    
If you do this successfully, you should see George W. Bush misrecognized as Colin Powell and Donald Rumsfeld correctly recognized.

In [ ]:
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/01/US_Navy_061108-F-5586B-137_President_George_W._Bush_looks_on_as_Secretary_of_Defense_Donald_H._Rumsfeld_addresses_the_nation_during_a_news_conference_from_the_Oval_Office.jpg/1280px-thumbnail.jpg'

# TODO: Your solution here




In [ ]:
#@title Task solution

url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/01/US_Navy_061108-F-5586B-137_President_George_W._Bush_looks_on_as_Secretary_of_Defense_Donald_H._Rumsfeld_addresses_the_nation_during_a_news_conference_from_the_Oval_Office.jpg/1280px-thumbnail.jpg'

# Load and display the image
image = load_image_from_url(url)
display_image(image)

# Detect, overlay and display faces in the image
face_detections = detect_faces_haar(image, haar_cascade_classifier, 1.3, 6)
overlaid_image = draw_rectangles_tlbr(image, face_detections, thickness=3)
display_image(overlaid_image)

# Iterate through all face detections
for tlx, tly, brx, bry in face_detections:
    face_crop = image[tly:bry, tlx:brx, :]

    # Determine the name belonging to the face
    name = recognize_face(face_crop, pca, knn, lfw_people_dataset.target_names)

    # Display the face along with the recognized name
    display_image(face_crop, name)

## 3.2 PCA Dimensions and K Neighbours Experimentation
In sections 1.2 and 1.3 of this lab, we arbitrarily chose to reduce the dataset to 100 dimensions and use 6 nearest neighbours for classification.

Your task is to experiment with changing the number of dimensions and nearest neighbours and evaluate how this impacts the recognition accuracy on the dataset.

Summarize your results in a table with columns for `PCA dimensions`, `K-Neighbours` and the macro avg `precision`, `recall` and `f1 score` (We generated this table in section 1.4). What combination gave you the best results?

# 4. Challenge Tasks
These tasks are meant to help pull together everything you have covered in this lab or extend on other exercises previously covered. It is highly recommended that you give these tasks a go, but only try to once you've finished the Lab Exercises section.

## Challenge 1 - Creating Your Own Face Recognition System
This challenge task should be seen as a potential idea for a project to further develop your understanding of the content in this lab. It is expected that this project could take multiple days to complete.

We saw in the Face Recognition section of this lab how we could take a dataset of cropped faces and using PCA for dimensionality reduction and KNN for classification, we could create a facial recognition system. Unfortunately, this system can only recognize 7 different faces.

Your task is to:
* Create your own image recognition task (e.g. to recognize your friends or members of your family)
* Construct a custom dataset of face images of those in your image recognition task
    * You can make use of the face detection approaches we have seen in this lab to take crops of faces from a larger image
* Using the PCA that was fit to the dataset in this lab:
    * Use PCA to reduce the dimensionality of your custom dataset
    * Fit your own KNN algorithm using your custom dataset
    * Evaluate the performance on your custom dataset
* Fit PCA to your custom dataset
    * Fit your own KNN algorithm using your custom dataset
    * Evaluate the performance on your custom dataset

You're likely to find that if you fit PCA to your custom dataset the results are better. This is expected if in general, the faces in your dataset look different than the faces in the dataset we explored in this lab.

Once you've done this, you will have a system that is able to recognize faces of those people who were in your custom dataset.

To extend this even further, you could write code to:
* Take an image
* Detect the faces in the image
* Crop out the faces and perform face recognition
* Overlay the face detection boxes and the name of the person who was recognized onto the original image
* Display the overlaid image

# Summary
In this lab we looked into face recognition using PCA and KNN, and saw how we could combine both face detection and face recognition into a single pipeline.